# **Evolution of Sentiment Analysis Models for Novels**

## **I. 감정 분석 모델의 발전 과정**

감정 분석 모델은 여러 학습 데이터 축적, 시행착오 등을 겪으며 최초의 모델인 1차 모델에서 최종 모델인 4차 모델로 발전해왔습니다. 이를 통해 소설에 대한 다양한 감정 분석 방법을 적용하도록 해주고, 문맥에 맞는 감정을 올바르게 분석할 수 있도록 점차 개선되었습니다.   
아래는 발전 과정을 담은 글입니다.



---



### **1차 MODEL**

#### **(1) 1차 MODEL의 주요 기능**
여러 시행착오와 기능들을 추가하여 탄생한 1차 모델은 소설의 감정 분석을 수행하고 시각화하며,  
이를 통한 후속 분석을 진원하는 기본적인 모델로,  주요 기능은 다음과 같습니다:

- 문장에 대한 감정 분석
 - Hugging Face Transformers 라이브러리를 사용하여 BERT 기반의 감정 분석 모델을 로드합니다.
 - 각 문장에 대해 감정 분석을 수행하고, 긍정적(POSITIVE) 감정은 양의 점수로, 부정적(NEGATIVE) 감정은 음의 점수로 변환하여 계산합니다.

- 세 가지 평균 계산 방법 이용
 - 각 문장의 감정 점수에 대해 세 가지 평균 계산 방법(단순 누적 평균, 가중 누적 평균, 이동 평균)을 통해 감정 그래프를 완성시키고 이를 그래프로 나타냅니다.
 - 이동 평균의 최댓값과 최솟값을 계산하고, 그래프에 표시하여 감정 극성 값의 범위를 알아볼 수 있도록 합니다.

- 감정 극성 변화 그래프 시각화
 - 이동 평균의 도함수를 구하여 감정 극성의 변화(volatility) 그래프를 시각화합니다.

- 함숫값의 차이를 이용한 유의미한 감정 변화 구간(Significant Different Point) 파악
 - 이동 평균과 가중 누적 평균 간의 차이를 계산하여, 특정 구간에서의 감정 점수 차이를 분석합니다. 이때 표준 편차를 기준으로 유의미한 차이가 발생하는 구간을 식별하고, 이는 감정의 변화가 급격히 일어나는 구간(Significant Different Point)임을 의미하므로 그 구간의 문장을 출력합니다.


#### **(2) 1차 MODEL의 코드**

아래는 1차 모델의 전체 코드 원본입니다. 하지만 이는 최종 코드가 아니므로 유심히 보지 않으셔도 됩니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
from transformers import pipeline

# 파일 경로 설정
file_path = '/경로/소설.txt'  # 텍스트 파일 경로 설정

# 텍스트 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# 텍스트를 문장 단위로 분할 (정규표현식 사용)
sentences = re.split(r'(?<=[.!?]) +', text)
sentences = [sentence for sentence in sentences if sentence]  # 빈 문장 제거

# 총 문장의 개수 계산
total_sentences = len(sentences)

# 감정 분석 모델 로드 (Hugging Face Transformers, BERT 기반 모델)
sentiment_analysis_pipeline = pipeline("sentiment-analysis")

# 감정 분석 수행 함수 정의
def get_sentiment_score(sentence):
    results = sentiment_analysis_pipeline(sentence)
    score = 0
    for result in results:
        label = result['label']
        if label == 'POSITIVE':
            score += result['score']
        elif label == 'NEGATIVE':
            score -= result['score']
    return score

# 각 문장에 대해 감정 점수 계산
sentiment_scores = [get_sentiment_score(sentence) for sentence in sentences]

# 단순 누적 평균 계산
cumulative_avg = np.cumsum(sentiment_scores) / (np.arange(total_sentences) + 1)

# 이동 평균 계산 함수 정의
def moving_average(data, window_size):
    return [np.mean(data[i:i + window_size]) for i in range(0, len(data), window_size)]

# 이동 평균 계산
window_size = 10
sentiment_avg = moving_average(sentiment_scores, window_size)

# 그래프 그리기
plt.figure(figsize=(12, 6))
plt.plot(cumulative_avg, label='Cumulative Average', color='blue')
plt.plot(np.arange(0, total_sentences, window_size), sentiment_avg, label='Moving Average', color='red')
plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Score')
plt.title('Sentiment Analysis')
plt.legend()
plt.show()

# 그래프 설정 및 도출
plt.figure(figsize=(12, 6))

# 단순 누적 평균 그래프
plt.plot(cumulative_avg, label='Cumulative Average', color='blue')

# 이동 평균 그래프
window_size = 10
sentiment_avg = moving_average(sentiment_scores, window_size)
plt.plot(np.arange(0, total_sentences, window_size), sentiment_avg, label='Moving Average', color='red')

plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Score')
plt.title('Sentiment Analysis')
plt.legend()
plt.show()

# 감정 극성의 변화 시각화
volatility = np.diff(sentiment_scores)
plt.figure(figsize=(12, 6))
plt.plot(volatility, label='Sentiment Volatility', color='green')
plt.axhline(0, color='grey', linestyle='--')
plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Change')
plt.title('Sentiment Volatility')
plt.legend()
plt.show()

# 이동 평균의 최대값과 최소값 계산
moving_avg_max = max(sentiment_avg)
moving_avg_min = min(sentiment_avg)

# 이동 평균의 최대값과 최소값을 그래프에 표시
plt.figure(figsize=(12, 6))
plt.plot(np.arange(0, total_sentences, window_size), sentiment_avg, label='Moving Average', color='red')
plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Score')
plt.title('Sentiment Analysis with Max/Min')
plt.axhline(moving_avg_max, color='green', linestyle='--', label=f'Max Moving Average: {moving_avg_max:.2f}')
plt.axhline(moving_avg_min, color='purple', linestyle='--', label=f'Min Moving Average: {moving_avg_min:.2f}')
plt.legend()
plt.show()

# 가중 누적 평균 (EWMA) 계산
def exponential_weighted_moving_average(data, alpha=0.3):
    ewma = []
    for i in range(len(data)):
        if i == 0:
            ewma.append(data[i])
        else:
            ewma.append(alpha * data[i] + (1 - alpha) * ewma[-1])
    return ewma

ewma = exponential_weighted_moving_average(sentiment_scores)

# 가중 누적 평균 (EWMA) 그래프
plt.figure(figsize=(12, 6))
plt.plot(cumulative_avg, label='Cumulative Average', color='blue')
plt.plot(np.arange(0, total_sentences, window_size), sentiment_avg, label='Moving Average', color='red')
plt.plot(ewma, label='Exponential Weighted Moving Average', color='orange')
plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Score')
plt.title('Sentiment Analysis with EWMA')
plt.legend()
plt.show()

# 특정 구간의 감정 점수 차이 분석
difference = np.abs(np.array(ewma) - np.array(sentiment_avg[:len(ewma)]))
std_dev_difference = np.std(difference)
threshold_difference = 1.5 * std_dev_difference

significant_diff_indices = np.where(difference >= threshold_difference)[0]

# 특정 구간의 문장 인덱스 및 출력
actual_indices = []
for idx in significant_diff_indices:
    start_idx = max(0, idx * window_size - window_size // 2)
    end_idx = min(total_sentences, idx * window_size + window_size // 2)
    actual_indices.append((start_idx, end_idx))

print("<Significant Different Point>")
for start_idx, end_idx in actual_indices:
    print(f"\nSentence Index: {start_idx} ~ {end_idx-1}\n")
    context_sentences = sentences[start_idx:end_idx]
    for sentence in context_sentences:
        print(sentence)
    print("\n---\n")



---



### **2차 MODEL**

#### **(1) 2차 MODEL의 개선 및 추가 기능**
1차 모델에서 2차 모델로 업데이트되면서 추가되거나, 개선된 부분은 다음과 같습니다:

- **감정 분석 모델 개선**
  - "nlptown/bert-base-multilingual-uncased-sentiment" 모델을 사용하여 다국어 감정 분석을 지원합니다.
  - 감정 레이블을 별점(1~5 stars)으로 세분화하여 더 정교한 감정 점수를 부여합니다.

- **specific_size와 standard_num 계산**
  - 텍스트의 총 문장 수에 따라 최적의 specific_size와 standard_num을 동적으로 계산합니다.

- **가우시안 스무딩 적용**
  - sigma 값을 문장의 수에 따라 동적으로 설정하여, 감정 점수의 스무딩(부드럽게) 처리를 적용합니다.

- **차이가 크게 나타나는 구간 식별 및 시각화**
  - 이동 평균과 가중 누적 평균 간의 차이가 크게 나타나는 구간을 식별하여 직사각형으로 강조 표시합니다.
  - 인접한 구간을 합쳐서 더 명확한 분석 결과를 제공합니다.

- **감정 변동성 분석**
  - 감정 변동성(volatility)을 이동 평균의 도함수를 사용하여 시각화하고, 곡선 길이를 계산하여 정규화된 arc length를 통해 감정 변동성을 정량화합니다.

- **양의 방향과 음의 방향 적분값 계산**
  - 가중 누적 평균 그래프를 적분하여, 긍정적 감정 점수와 부정적 감정 점수를 각각 계산하고 이를 시각화합니다.

- **막대 바 차트 시각화**
  - 긍정적 감정 점수와 부정적 감정 점수를 막대 바 차트로 시각화하여 전체 소설의 감정 분석 결과를 한눈에 볼 수 있도록 합니다.

#### **(2) 2차 MODEL의 코드**

아래는 1차 모델의 전체 코드 원본입니다. 이 또한 최종 코드가 아니므로 유심히 보지 않으셔도 됩니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
from transformers import pipeline

# 파일 경로 설정
file_path = '/경로/소설.txt'

# 텍스트 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# 텍스트를 문장 단위로 분할
sentences = re.split(r'(?<=[.!?]) +', text)
sentences = [sentence for sentence in sentences if sentence]  # 빈 문장 제거

# 총 문장의 개수 계산
total_sentences = len(sentences)

# 감정 분석 모델 로드
sentiment_analysis_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# 감정 분석 수행 함수 정의
def get_sentiment_score(sentence):
    results = sentiment_analysis_pipeline(sentence)
    label = results[0]['label']
    if label == '1 star':
        return -1
    elif label == '2 stars':
        return -0.5
    elif label == '3 stars':
        return 0
    elif label == '4 stars':
        return 0.5
    elif label == '5 stars':
        return 1

# 모든 문장에 대한 감정 분석 수행
sentiment_analysis = [get_sentiment_score(sentence) for sentence in sentences]

# specific_size와 standard_num 계산 함수
def calculate_specific_size(total_sentences, target_standard_num=20):
    best_specific_size = None
    closest_difference = float('inf')
    best_standard_num = None
    for specific_size in range(1, total_sentences + 1):
        standard_num = total_sentences // specific_size
        difference = abs(standard_num - target_standard_num)
        if difference < closest_difference:
            closest_difference = difference
            best_specific_size = specific_size
            best_standard_num = standard_num
        if closest_difference == 0:
            break
    return best_specific_size, best_standard_num

specific_size, standard_num = calculate_specific_size(total_sentences)

# 단순 누적 평균 계산
cumulative_avg = np.cumsum(sentiment_analysis) / (np.arange(total_sentences) + 1)

# 이동 평균 계산 함수 정의
def moving_average(data, window_size):
    return [np.mean(data[i:i + window_size]) for i in range(0, len(data), window_size)]

# 세 가지 방법 모두 specific_size 만큼의 평균으로 통일
sentiment_avg = moving_average(sentiment_analysis, specific_size)
cumulative_avg_avg = moving_average(cumulative_avg, specific_size)
ewma = pd.Series(sentiment_analysis).ewm(span=specific_size, adjust=False).mean()
ewma_avg = moving_average(ewma, specific_size)

# 이동 평균의 최대값과 최소값 계산
moving_avg_max = max(sentiment_avg)
moving_avg_min = min(sentiment_avg)

# sigma 값을 동적으로 설정
sigma_value = max(0.1, min(1.0, 10.0 / np.sqrt(total_sentences)))  # 문장의 수에 따라 sigma 값을 조정
sentiment_avg_smooth = gaussian_filter1d(sentiment_avg, sigma=sigma_value)
cumulative_avg_smooth = gaussian_filter1d(cumulative_avg_avg, sigma=sigma_value)
ewma_avg_smooth = gaussian_filter1d(ewma_avg, sigma=sigma_value)

# X 좌표 설정
x_sentiment = np.arange(len(sentiment_avg)) * specific_size
x_cumulative = np.arange(len(cumulative_avg_avg)) * specific_size
x_ewma = np.arange(len(ewma_avg)) * specific_size

# 차이가 크게 나타나는 구간 식별
difference = np.abs(np.array(ewma_avg) - np.array(sentiment_avg))
std_dev_difference = np.std(difference)
threshold_difference = 1.5 * std_dev_difference  # 표준 편차의 1.5배를 임계값으로 설정

significant_diff_indices = np.where(difference >= threshold_difference)[0]

# 오차 범위를 고려한 실제 문장 인덱스 계산
actual_indices = []
for idx in significant_diff_indices:
    start_idx = max(0, idx * specific_size - specific_size // 2)
    end_idx = min(total_sentences, idx * specific_size + specific_size // 2)
    actual_indices.append((start_idx, end_idx))

# 인접한 구간 합치기
merged_indices = []
for start_idx, end_idx in actual_indices:
    if not merged_indices:
        merged_indices.append([start_idx, end_idx])
    else:
        last_start, last_end = merged_indices[-1]
        if start_idx <= last_end:
            merged_indices[-1] = [last_start, max(last_end, end_idx)]
        else:
            merged_indices.append([start_idx, end_idx])

# 그래프 그리기
plt.figure(figsize=(12, 6))

# 단순 누적 평균
plt.plot(x_cumulative, cumulative_avg_smooth, color='#DB4455', linestyle='--', label='Cumulative Average')

# 가중 누적 평균 (EWMA)
plt.plot(x_ewma, ewma_avg_smooth, color='#0E5CAD', linestyle='-', label='Exponential Weighted Moving Average')

# 이동 평균
plt.plot(x_sentiment, sentiment_avg_smooth, color='#623AA2', linestyle='-', label='Moving Average')

# 차이가 크게 나타나는 구간에 직사각형 추가
for start_idx, end_idx in merged_indices:
    end_idx = min(end_idx, total_sentences)  # 소설의 끝을 넘지 않도록 조정
    plt.axvspan(start_idx, end_idx, color='lightgrey', alpha=0.5)

plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Polarity (-1 to 1)')
plt.title('Sentiment Analysis using Different Calculating Methods')
plt.axhline(0, color='grey', linestyle='--')  # 중립 감정선
plt.legend()

# 이동 평균의 최대값과 최소값을 그래프에 표시
plt.text(0.95, 0.10, f"max = {moving_avg_max:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')
plt.text(0.95, 0.05, f"min = {moving_avg_min:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

# 감정 변동성 분석 (이동 평균의 도함수 사용)
volatility = np.diff(sentiment_avg_smooth)

# 가우시안 스무딩 적용 (곡률을 더 크게)
volatility_smooth = gaussian_filter1d(volatility, sigma=2)

# 감정 변동성 시각화 및 곡선 길이 계산
plt.figure(figsize=(12, 6))
plt.plot(np.arange(len(volatility_smooth)), volatility_smooth, color='#B4C99D', linestyle='-')
plt.axhline(0, color='grey', linestyle='--')  # y=0의 값에 회색 점선 추가
plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Change')
plt.title('Sentiment Volatility')

# 곡선 길이 계산 및 정규화된 arc length 계산
compressed_x_volatility = np.linspace(0, total_sentences, num=100)
compressed_volatility_smooth = np.interp(compressed_x_volatility, np.linspace(0, total_sentences, num=len(volatility_smooth)), volatility_smooth)
arc_length = np.sum(np.sqrt(np.diff(compressed_x_volatility)**2 + np.diff(compressed_volatility_smooth)**2))

# 전체 길이 계산
total_length = np.linalg.norm([compressed_x_volatility[-1] - compressed_x_volatility[0], compressed_volatility_smooth[-1] - compressed_volatility_smooth[0]])

# 소설 길이 기반 정규화
novel_length = total_sentences  # 소설의 전체 문장 수
normalized_arc_length = arc_length / novel_length

# 상대적 곡선 길이 계산
relative_arc_length = normalized_arc_length / total_length

# 스케일링 적용
scaled_relative_arc_length = relative_arc_length * 100  # 예를 들어 0에서 100 사이로 스케일링

# 길이 점수 출력 (그래프의 '네모 박스'의 우측 하단에 표시)
plt.text(0.95, 0.05, f"length score = {scaled_relative_arc_length:.2f}", fontsize=12, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

# 실제 문장 인덱스 기반 텍스트 분석
print("<Significant Different Point>")
for start_idx, end_idx in merged_indices:
    end_idx = min(end_idx, total_sentences)  # 소설의 끝을 넘지 않도록 조정
    print(f"\nSentence Index: {start_idx} ~ {end_idx-1}\n")
    context_sentences = sentences[start_idx:end_idx]
    for sentence in context_sentences:
        sentence = sentence.replace('\n', ' ')
        print(f"{sentence}")
    print("\n---\n")

# 가중 누적 평균 그래프 적분을 위한 데이터 압축
compressed_x = np.linspace(0, 100, num=100)
compressed_y = np.interp(compressed_x, np.linspace(0, 100, num=len(ewma)), ewma)

# 양의 방향과 음의 방향 적분값 계산
positive_integral = np.trapz(compressed_y[compressed_y > 0], compressed_x[compressed_y > 0])
negative_integral = np.trapz(compressed_y[compressed_y < 0], compressed_x[compressed_y < 0])

# 결과를 100점 만점으로 환산
positive_score = positive_integral
negative_score = abs(negative_integral)

# 막대 바 차트 시각화
fig, ax = plt.subplots(figsize=(8, 6))

# 데이터 준비
categories = ['Positive Sentiment Score', 'Negative Sentiment Score']
scores = [positive_score, negative_score]
colors = ["#DC143C", "#392F31"]

# 막대 바 차트 그리기
bars = plt.bar(categories, scores, color=colors)

# 레이블 추가 (막대 바 위쪽에 위치하도록)
for bar, score in zip(bars, scores):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, yval + 0.1, f'{score:.1f} points', ha='center', va='bottom', color='black')

# y축 최대값 설정
max_score = max(positive_score, negative_score)
ymax = np.ceil(max_score * 1.2)

# y축에 숫자를 표시한 지점마다 x축과 평행한 점선 추가
ax.yaxis.grid(True, which='both', color='lightgrey', linestyle='--', linewidth=0.7)
ax.set_axisbelow(True)
ax.set_ylim(0, ymax)

# 제목 설정
plt.title('Overall Sentiment Analysis of the Novel')

plt.show()



---



### **3차 MODEL**

#### **(1) 3차 MODEL의 개선 및 추가 기능**
2차 모델에서 3차 모델로 업그레이드되면서 추가되거나, 개선된 부분은 다음과 같습니다:

- **입력 길이 제한 추가**
  - 최대 입력 길이(MAX_LENGTH)를 512로 설정하여, 감정 분석을 위한 입력 문장의 길이를 제한합니다.

- **이전 문장 감정 점수 반영**
  - 감정 점수 계산 시 이전 문장의 감정 점수를 반영하여, 문맥에 따른 감정 변화를 더 정확히 분석합니다. 예를 들어, '3 stars' 레이블의 경우 이전 문장이 긍정적이면 0.1 점수를 더하고, 부정적이면 0.1 점수를 뺍니다.

- **가우시안 스무딩 개선**
  - sigma 값을 동적으로 설정하여 문장의 수에 따라 감정 점수를 더 부드럽게 처리하고, 감정 점수의 변동을 더 정확히 반영합니다.

- **곡선 길이 계산 오류 수정**
  - 곡선 길이 계산 및 정규화된 arc length 계산 시 발생할 수 있는 오류를 수정하여, 더 정확한 감정 변동성 분석을 수행합니다.

- **막대 바 차트 시각화 수정**
  - 감정 분석 결과를 더 명확히 시각화하기 위해 막대 바 차트를 수정하고, 레이블을 추가하여 긍정적 감정 점수와 부정적 감정 점수를 각각 시각적으로 비교할 수 있도록 개선합니다.

#### **(2) 3차 MODEL의 코드**

3차 모델의 코드입니다. 이 또한 최종 코드와 다르므로 유심히 보지 않으셔도 좋습니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from transformers import pipeline

# 파일 경로 설정
file_path = '/경로/소설.txt'  # 임의로 바꿔서 사용하면 된다.

# 텍스트 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# 텍스트를 문장 단위로 분할 (정규표현식 사용)
sentences = re.split(r'(?<=[.!?]) +', text)
sentences = [sentence for sentence in sentences if sentence]  # 빈 문장 제거

# 총 문장의 개수 계산
total_sentences = len(sentences)

# 감정 분석 모델 로드 (Hugging Face Transformers, BERT 기반 모델)
sentiment_analysis_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# 최대 입력 길이 설정
MAX_LENGTH = 512

# 감정 분석 수행 함수 정의
def get_sentiment_score(sentence, prev_score=None):
    if len(sentence) > MAX_LENGTH:
        sentence = sentence[:MAX_LENGTH]  # 입력 문장을 최대 길이로 자르기
    results = sentiment_analysis_pipeline(sentence)

    # 감정 점수 통합
    score = 0
    for result in results:
        label = result['label']
        probability = result['score']

        if label == '1 star':
            score -= 1 * probability
        elif label == '2 stars':
            score -= 0.5 * probability
        elif label == '3 stars':
            if prev_score is not None:
                if prev_score > 0:
                    score += 0.1 * probability  # 이전 문장이 긍정적일 경우
                else:
                    score -= 0.1 * probability  # 이전 문장이 부정적일 경우
            # 이전 문장이 없을 경우 (첫 번째 문장)
        elif label == '4 stars':
            score += 0.5 * probability
        elif label == '5 stars':
            score += 1 * probability

    return score

# 첫 번째 문장은 이전 문장이 없으므로 먼저 처리
sentiment_analysis = [get_sentiment_score(sentences[0])]

# 나머지 문장 처리
for i in range(1, total_sentences):
    prev_score = sentiment_analysis[-1]
    sentiment_analysis.append(get_sentiment_score(sentences[i], prev_score))

# specific_size와 standard_num 계산
def calculate_specific_size(total_sentences, target_standard_num=20):
    best_specific_size = None
    closest_difference = float('inf')
    best_standard_num = None

    for specific_size in range(1, total_sentences + 1):
        standard_num = total_sentences // specific_size
        difference = abs(standard_num - target_standard_num)

        if difference < closest_difference:
            closest_difference = difference
            best_specific_size = specific_size
            best_standard_num = standard_num

        if closest_difference == 0:
            break

    return best_specific_size, best_standard_num

specific_size, standard_num = calculate_specific_size(total_sentences)

# 단순 누적 평균 계산
cumulative_avg = np.cumsum(sentiment_analysis) / (np.arange(total_sentences) + 1)

# 가중 누적 평균 계산 (EWMA)
ewma = pd.Series(sentiment_analysis).ewm(span=specific_size, adjust=False).mean()

# 이동 평균 계산 함수 정의
def moving_average(data, window_size):
    return [np.mean(data[i:i + window_size]) for i in range(0, len(data), window_size)]

# 세 가지 방법 모두 specific_size 만큼의 평균으로 통일
sentiment_avg = moving_average(sentiment_analysis, specific_size)
cumulative_avg_avg = moving_average(cumulative_avg, specific_size)
ewma_avg = moving_average(ewma, specific_size)

# moving average의 최대값과 최소값 계산
moving_avg_max = max(sentiment_avg)
moving_avg_min = min(sentiment_avg)

# sigma 값을 동적으로 설정
sigma_value = max(0.1, min(1.0, 10.0 / np.sqrt(total_sentences)))  # 문장의 수에 따라 sigma 값을 조정
sentiment_avg_smooth = gaussian_filter1d(sentiment_avg, sigma=sigma_value)
cumulative_avg_smooth = gaussian_filter1d(cumulative_avg_avg, sigma=sigma_value)
ewma_avg_smooth = gaussian_filter1d(ewma_avg, sigma=sigma_value)

# X 좌표 설정
x_sentiment = np.arange(len(sentiment_avg)) * specific_size
x_cumulative = np.arange(len(cumulative_avg_avg)) * specific_size
x_ewma = np.arange(len(ewma_avg)) * specific_size

# 차이가 크게 나타나는 구간 식별
difference = np.abs(np.array(ewma_avg) - np.array(sentiment_avg))
std_dev_difference = np.std(difference)
threshold_difference = 1.5 * std_dev_difference  # 표준 편차의 1.5배를 임계값으로 설정

significant_diff_indices = np.where(difference >= threshold_difference)[0]

# 오차 범위를 고려한 실제 문장 인덱스 계산
actual_indices = []
for idx in significant_diff_indices:
    start_idx = max(0, idx * specific_size - specific_size // 2)
    end_idx = min(total_sentences, idx * specific_size + specific_size // 2)
    actual_indices.append((start_idx, end_idx))

# 인접한 구간 합치기
merged_indices = []
for start_idx, end_idx in actual_indices:
    if not merged_indices:
        merged_indices.append([start_idx, end_idx])
    else:
        last_start, last_end = merged_indices[-1]
        if start_idx <= last_end:
            merged_indices[-1] = [last_start, max(last_end, end_idx)]
        else:
            merged_indices.append([start_idx, end_idx])

# 그래프 그리기
plt.figure(figsize=(12, 6))

# 단순 누적 평균
plt.plot(x_cumulative, cumulative_avg_smooth, color='#DB4455', linestyle='--', label='Cumulative Average')

# 가중 누적 평균 (EWMA)
plt.plot(x_ewma, ewma_avg_smooth, color='#0E5CAD', linestyle='-', label='Exponential Weighted Moving Average')

# 이동 평균
plt.plot(x_sentiment, sentiment_avg_smooth, color='#623AA2', linestyle='-', label='Moving Average')

# 차이가 크게 나타나는 구간에 직사각형 추가
for start_idx, end_idx in merged_indices:
    end_idx = min(end_idx, total_sentences)  # 소설의 끝을 넘지 않도록 조정
    plt.axvspan(start_idx, end_idx, color='lightgrey', alpha=0.5)

plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Polarity (-1 to 1)')
plt.title('Sentiment Analysis using Different Calculating Methods')
plt.axhline(0, color='grey', linestyle='--')  # 중립 감정선
plt.legend()

# 이동 평균의 최대값과 최소값을 그래프에 표시
plt.text(0.95, 0.10, f"max = {moving_avg_max:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')
plt.text(0.95, 0.05, f"min = {moving_avg_min:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

total_length = np.linalg.norm([compressed_x_volatility[-1] - compressed_x_volatility[0], compressed_volatility_smooth[-1] - compressed_volatility_smooth[0]])

# 소설 길이 기반 정규화
novel_length = total_sentences  # 소설의 전체 문장 수
normalized_arc_length = arc_length / novel_length

# 상대적 곡선 길이 계산
relative_arc_length = normalized_arc_length / total_length

# 스케일링 적용
scaled_relative_arc_length = relative_arc_length * 100  # 예를 들어 0에서 100 사이로 스케일링

# 길이 점수 출력 (그래프의 '네모 박스'의 우측 하단에 표시)
plt.text(0.95, 0.05, f"length score = {scaled_relative_arc_length:.2f}", fontsize=12, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

# 실제 문장 인덱스 기반 텍스트 분석
print("<Significant Different Point>")
for start_idx, end_idx in merged_indices:
    end_idx = min(end_idx, total_sentences)  # 소설의 끝을 넘지 않도록 조정
    print(f"\nSentence Index: {start_idx} ~ {end_idx-1}\n")
    context_sentences = sentences[start_idx:end_idx]
    for sentence in context_sentences:
        sentence = sentence.replace('\n', ' ')
        print(f"{sentence}")
    print("\n---\n")

# 가중 누적 평균 그래프 적분을 위한 데이터 압축
compressed_x = np.linspace(0, 100, num=100)
compressed_y = np.interp(compressed_x, np.linspace(0, 100, num=len(ewma)), ewma)

# 양의 방향과 음의 방향 적분값 계산
positive_integral = np.trapz(compressed_y[compressed_y > 0], compressed_x[compressed_y > 0])
negative_integral = np.trapz(compressed_y[compressed_y < 0], compressed_x[compressed_y < 0])

# 결과를 100점 만점으로 환산
positive_score = positive_integral
negative_score = abs(negative_integral)

# 막대 바 차트 시각화 수정된 코드
fig, ax = plt.subplots(figsize=(8, 6))

# 데이터 준비
categories = ['Positive Sentiment Score', 'Negative Sentiment Score']
scores = [positive_score, negative_score]
colors = ["#DC143C", "#392F31"]

# 막대 바 차트 그리기
bars = plt.bar(categories, scores, color=colors)

# 레이블 추가 (막대 바 위쪽에 위치하도록)
for bar, score in zip(bars, scores):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, yval + 0.1, f'{score:.1f} points', ha='center', va='bottom', color='black')

# y축 최대값 설정
max_score = max(positive_score, negative_score)
ymax = np.ceil(max_score * 1.2)

# y축에 숫자를 표시한 지점마다 x축과 평행한 점선 추가
ax.yaxis.grid(True, which='both', color='lightgrey', linestyle='--', linewidth=0.7)
ax.set_axisbelow(True)
ax.set_ylim(0, ymax)

# 제목 설정
plt.title('Overall Sentiment Analysis of the Novel')

plt.show()



---



### **4차 MODEL**

#### **(1) 4차 MODEL의 개선 및 추가 기능**
3차 모델에서 4차 모델로 업그레이드되면서 추가되거나, 개선된 부분은 다음과 같습니다:

- **다중 감정 인식 및 점수 할당**
  - 감정 분석 시 'anger', 'disgust', 'fear', 'joy', 'sadness'의 다중 감정을 인식하고, 각각의 감정에 대한 점수를 할당합니다. 이는 별점(1~5 stars) 레이블을 기반으로 감정 점수를 세분화하여 할당합니다.

- **긍정적 및 부정적 문맥에 따른 가중치 적용**
  - 문장 내 긍정적 또는 부정적 단어를 감지하여, 문맥에 따른 가중치를 적용합니다. 예를 들어, 긍정적 단어가 포함된 문장은 1.2의 가중치를, 부정적 단어가 포함된 문장은 0.8의 가중치를 부여합니다.

- **감정 점수 통합 방식 개선**
  - 감정 점수 통합 시 각 감정의 점수에 가중치를 부여하여, 더 정교한 감정 점수를 계산합니다. 예를 들어, 'anger'는 -1, 'disgust'는 -0.5, 'fear'는 -0.7, 'joy'는 1, 'sadness'는 -1의 가중치를 부여합니다.

- **감정 변동성 분석 개선**
  - 감정 변동성(volatility) 분석 시 가우시안 스무딩을 적용하여 곡률을 더 크게 하여, 감정 변화의 세부적인 부분을 더 정확히 시각화합니다.
  - x축을 문장의 수에 맞게 조정하여, 시각화된 그래프가 문장의 실제 수와 일치하도록 개선합니다.

- **실제 문장 인덱스 기반 텍스트 분석 개선**
  - 차이가 크게 나타나는 구간을 식별하여, 해당 구간의 문장을 출력합니다. 이 과정에서 소설의 끝을 넘지 않도록 조정하여 더 정확한 분석을 제공합니다.

- **막대 바 차트 시각화 개선**
  - 긍정적 감정 점수와 부정적 감정 점수를 시각적으로 비교하기 위해 막대 바 차트를 수정하고, 더 명확한 레이블을 추가합니다.
  - y축 최대값을 설정하고, y축에 숫자를 표시한 지점마다 x축과 평행한 점선을 추가하여 시각적으로 더 명확한 정보를 제공합니다.

- **length score로 변경**
  - 감정 변동성 분석을 위해 arc length를 기반으로 length score를 계산하여, 더 직관적으로 감정 변화의 정도를 표현합니다.

#### **(2) 4차 MODEL의 코드**

3차 모델의 코드입니다. 이 또한 최종 코드와 다르므로 유심히 보지 않으셔도 좋습니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from transformers import pipeline

# 파일 경로 설정
file_path = '/경로/소설.txt'  # 임의로 바꿔서 사용하면 된다.

# 텍스트 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# 텍스트를 문장 단위로 분할 (정규표현식 사용)
sentences = re.split(r'(?<=[.!?]) +', text)
sentences = [sentence for sentence in sentences if sentence]  # 빈 문장 제거

# 총 문장의 개수 계산
total_sentences = len(sentences)

# 감정 분석 모델 로드 (Hugging Face Transformers, BERT 기반 모델)
sentiment_analysis_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# 최대 입력 길이 설정
MAX_LENGTH = 512

# 감정 분석 수행 함수 정의
def get_sentiment_score(sentence, prev_score=None):
    if len(sentence) > MAX_LENGTH:
        sentence = sentence[:MAX_LENGTH]  # 입력 문장을 최대 길이로 자르기
    results = sentiment_analysis_pipeline(sentence)

    # 다중 감정 인식 및 점수 할당
    emotion_scores = {'anger': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0}
    for result in results:
        label = result['label']
        probability = result['score']

        if label == '1 star':
            emotion_scores['sadness'] += probability
        elif label == '2 stars':
            emotion_scores['fear'] += probability
        elif label == '3 stars':
            emotion_scores['disgust'] += probability
        elif label == '4 stars':
            emotion_scores['joy'] += probability
        elif label == '5 stars':
            emotion_scores['joy'] += 2 * probability

    # 감정 점수 통합
    score = (
        -1 * emotion_scores['anger'] +
        -0.5 * emotion_scores['disgust'] +
        -0.7 * emotion_scores['fear'] +
        1 * emotion_scores['joy'] +
        -1 * emotion_scores['sadness']
    )

    return score

# 긍정적 및 부정적 단어 리스트 정의
positive_words = [
    "happy", "joyful", "excellent", "good", "wonderful", "amazing", "great", "positive", "fortunate", "pleased",
    "delightful", "content", "cheerful", "satisfied", "thrilled", "ecstatic", "merry", "elated", "jubilant", "gleeful"
]

negative_words = [
    "sad", "terrible", "bad", "awful", "horrible", "negative", "unfortunate", "displeased", "angry", "fearful",
    "miserable", "depressed", "unhappy", "downcast", "gloomy", "dismal", "wretched", "heartbroken", "melancholy", "despair"
]

# 감정 분석 수행 함수 정의
def get_sentiment_score(sentence, prev_score=None):
    if len(sentence) > MAX_LENGTH:
        sentence = sentence[:MAX_LENGTH]  # 입력 문장을 최대 길이로 자르기
    results = sentiment_analysis_pipeline(sentence)

    # 문맥에 따라 긍정 또는 부정 가중치 적용
    context_weight = 1.0
    if any(word in sentence for word in positive_words):
        context_weight = 1.2  # 긍정 문맥 가중치
    elif any(word in sentence for word in negative_words):
        context_weight = 0.8  # 부정 문맥 가중치

    # 다중 감정 인식 및 점수 할당
    emotion_scores = {'anger': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0}
    for result in results:
        label = result['label']
        probability = result['score']

        if label == '1 star':
            emotion_scores['sadness'] += probability
        elif label == '2 stars':
            emotion_scores['fear'] += probability
        elif label == '3 stars':
            emotion_scores['disgust'] += probability
        elif label == '4 stars':
            emotion_scores['joy'] += probability
        elif label == '5 stars':
            emotion_scores['joy'] += 2 * probability

    # 감정 점수 통합 및 가중치 적용
    score = (
        -1 * emotion_scores['anger'] +
        -0.5 * emotion_scores['disgust'] +
        -0.7 * emotion_scores['fear'] +
        1 * emotion_scores['joy'] +
        -1 * emotion_scores['sadness']
    ) * context_weight

    return score

# 첫 번째 문장은 이전 문장이 없으므로 먼저 처리
sentiment_analysis = [get_sentiment_score(sentences[0])]

# 나머지 문장 처리
for i in range(1, total_sentences):
    prev_score = sentiment_analysis[-1]
    sentiment_analysis.append(get_sentiment_score(sentences[i], prev_score))

# specific_size와 standard_num 계산
def calculate_specific_size(total_sentences, target_standard_num=20):
    best_specific_size = None
    closest_difference = float('inf')
    best_standard_num = None

    for specific_size in range(1, total_sentences + 1):
        standard_num = total_sentences // specific_size
        difference = abs(standard_num - target_standard_num)

        if difference < closest_difference:
            closest_difference = difference
            best_specific_size = specific_size
            best_standard_num = standard_num

        if closest_difference == 0:
            break

    return best_specific_size, best_standard_num

specific_size, standard_num = calculate_specific_size(total_sentences)

# 단순 누적 평균 계산
cumulative_avg = np.cumsum(sentiment_analysis) / (np.arange(total_sentences) + 1)

# 가중 누적 평균 계산 (EWMA)
ewma = pd.Series(sentiment_analysis).ewm(span=specific_size, adjust=False).mean()

# 이동 평균 계산 함수 정의
def moving_average(data, window_size):
    return [np.mean(data[i:i + window_size]) for i in range(0, len(data), window_size)]

# 세 가지 방법 모두 specific_size 만큼의 평균으로 통일
sentiment_avg = moving_average(sentiment_analysis, specific_size)
cumulative_avg_avg = moving_average(cumulative_avg, specific_size)
ewma_avg = moving_average(ewma, specific_size)

# moving average의 최대값과 최소값 계산
moving_avg_max = max(sentiment_avg)
moving_avg_min = min(sentiment_avg)

# sigma 값을 동적으로 설정
sigma_value = max(0.1, min(1.0, 10.0 / np.sqrt(total_sentences)))  # 문장의 수에 따라 sigma 값을 조정
sentiment_avg_smooth = gaussian_filter1d(sentiment_avg, sigma=sigma_value)
cumulative_avg_smooth = gaussian_filter1d(cumulative_avg_avg, sigma=sigma_value)
ewma_avg_smooth = gaussian_filter1d(ewma_avg, sigma=sigma_value)

# X 좌표 설정
x_sentiment = np.arange(len(sentiment_avg)) * specific_size
x_cumulative = np.arange(len(cumulative_avg_avg)) * specific_size
x_ewma = np.arange(len(ewma_avg)) * specific_size

# 차이가 크게 나타나는 구간 식별
difference = np.abs(np.array(ewma_avg) - np.array(sentiment_avg))
std_dev_difference = np.std(difference)
threshold_difference = 1.5 * std_dev_difference  # 표준 편차의 1.5배를 임계값으로 설정

significant_diff_indices = np.where(difference >= threshold_difference)[0]

# 오차 범위를 고려한 실제 문장 인덱스 계산
actual_indices = []
for idx in significant_diff_indices:
    start_idx = max(0, idx * specific_size - specific_size // 2)
    end_idx = min(total_sentences, idx * specific_size + specific_size // 2)
    actual_indices.append((start_idx, end_idx))

# 인접한 구간 합치기
merged_indices = []
for start_idx, end_idx in actual_indices:
    if not merged_indices:
        merged_indices.append([start_idx, end_idx])
    else:
        last_start, last_end = merged_indices[-1]
        if start_idx <= last_end:
            merged_indices[-1] = [last_start, max(last_end, end_idx)]
        else:
            merged_indices.append([start_idx, end_idx])

# 그래프 그리기
plt.figure(figsize=(12, 6))

# 단순 누적 평균
plt.plot(x_cumulative, cumulative_avg_smooth, color='#DB4455', linestyle='--', label='Cumulative Average')

# 가중 누적 평균 (EWMA)
plt.plot(x_ewma, ewma_avg_smooth, color='#0E5CAD', linestyle='-', label='Exponential Weighted Moving Average')

# 이동 평균
plt.plot(x_sentiment, sentiment_avg_smooth, color='#623AA2', linestyle='-', label='Moving Average')

# 차이가 크게 나타나는 구간에 직사각형 추가
for start_idx, end_idx in merged_indices:
    end_idx = min(end_idx, total_sentences)  # 소설의 끝을 넘지 않도록 조정
    plt.axvspan(start_idx, end_idx, color='lightgrey', alpha=0.5)

plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Polarity (-1 to 1)')
plt.title('Sentiment Analysis using Different Calculating Methods')
plt.axhline(0, color='grey', linestyle='--')  # 중립 감정선
plt.legend()

# 이동 평균의 최대값과 최소값을 그래프에 표시
plt.text(0.95, 0.10, f"max = {moving_avg_max:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')
plt.text(0.95, 0.05, f"min = {moving_avg_min:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

# 감정 변동성 분석 (이동 평균의 도함수 사용)
volatility = np.diff(sentiment_avg_smooth)

# 가우시안 스무딩 적용 (곡률을 더 크게)
volatility_smooth = gaussian_filter1d(volatility, sigma=2)

# 감정 변동성 시각화 및 곡선 길이 계산
plt.figure(figsize=(12, 6))

# x축을 문장의 수에 맞게 조정
plt.plot(np.linspace(0, total_sentences, len(volatility_smooth)), volatility_smooth, color='#B4C99D', linestyle='-')
plt.axhline(0, color='grey', linestyle='--')  # y=0의 값에 회색 점선 추가
plt.xlabel('Sentence Number')
plt.ylabel('Sentiment Change')
plt.title('Sentiment Volatility')

# 곡선 길이 계산 및 정규화된 arc length 계산
compressed_x_volatility = np.linspace(0, total_sentences, num=100)
compressed_volatility_smooth = np.interp(compressed_x_volatility, np.linspace(0, total_sentences, num=len(volatility_smooth)), volatility_smooth)
arc_length = np.sum(np.sqrt(np.diff(compressed_x_volatility)**2 + np.diff(compressed_volatility_smooth)**2))

# 전체 길이 계산
total_length = np.linalg.norm([compressed_x_volatility[-1] - compressed_x_volatility[0], compressed_volatility_smooth[-1] - compressed_volatility_smooth[0]])

# 소설 길이 기반 정규화
novel_length = total_sentences  # 소설의 전체 문장 수
normalized_arc_length = arc_length / novel_length

# 상대적 곡선 길이 계산
relative_arc_length = normalized_arc_length / total_length

# 스케일링 적용
scaled_relative_arc_length = relative_arc_length * 100000

# 길이 점수 출력 (그래프의 '네모 박스'의 우측 하단에 표시)
plt.text(0.95, 0.05, f"length score = {scaled_relative_arc_length:.2f}", fontsize=12, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

# 실제 문장 인덱스 기반 텍스트 분석
print("<Significant Different Point>")
for start_idx, end_idx in merged_indices:
    end_idx = min(end_idx, total_sentences)  # 소설의 끝을 넘지 않도록 조정
    print(f"\nSentence Index: {start_idx} ~ {end_idx-1}\n")
    context_sentences = sentences[start_idx:end_idx]
    for sentence in context_sentences:
        sentence = sentence.replace('\n', ' ')
        print(f"{sentence}")
    print("\n---\n")

# 가중 누적 평균 그래프 적분을 위한 데이터 압축
compressed_x = np.linspace(0, 100, num=100)
compressed_y = np.interp(compressed_x, np.linspace(0, 100, num=len(ewma)), ewma)

# 양의 방향과 음의 방향 적분값 계산
positive_integral = np.trapz(compressed_y[compressed_y > 0], compressed_x[compressed_y > 0])
negative_integral = np.trapz(compressed_y[compressed_y < 0], compressed_x[compressed_y < 0])

# 결과를 100점 만점으로 환산
positive_score = positive_integral
negative_score = abs(negative_integral)

# 막대 바 차트 시각화 수정된 코드
fig, ax = plt.subplots(figsize=(8, 6))

# 데이터 준비
categories = ['Positive Sentiment Score', 'Negative Sentiment Score']
scores = [positive_score, negative_score]
colors = ["#DC143C", "#392F31"]

# 막대 바 차트 그리기
bars = plt.bar(categories, scores, color=colors)

# 레이블 추가 (막대 바 위쪽에 위치하도록)
for bar, score in zip(bars, scores):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, yval + 0.1, f'{score:.1f} points', ha='center', va='bottom', color='black')

# y축 최대값 설정
max_score = max(positive_score, negative_score)
ymax = np.ceil(max_score * 1.2)

# y축에 숫자를 표시한 지점마다 x축과 평행한 점선 추가
ax.yaxis.grid(True, which='both', color='lightgrey', linestyle='--', linewidth=0.7)
ax.set_axisbelow(True)
ax.set_ylim(0, ymax)

# 제목 설정
plt.title('Overall Sentiment Analysis of the Novel')

plt.show()



---



### **5차 MODEL**

#### **(1) 기존 MODEL의 한계점**
4차 모델에서 발견된 주요 한계점은 다음과 같습니다:

- **감정 극성의 긍정 편향**
  - 4차 모델은 감정 극성이 긍정적으로 편향되는 경향이 있습니다. 이는 문맥에 따라 긍정적 단어에 더 높은 가중치가 부여되기 때문일 수 있습니다. 이로 인해 소설 내 부정적인 감정을 충분히 반영하지 못할 수 있습니다.

- **소설 극초반부에 EWMA 값의 불규칙한 변동**
  - EWMA(가중 이동 평균) 계산 시, 소설 초반부에 값이 불규칙하게 튀는 현상이 나타납니다. 이는 초기값 설정 문제로 인해 발생할 수 있습니다. 이러한 변동은 분석의 신뢰성을 저하시킬 수 있습니다.

- **length score의 비합리적 계산**
  - length score는 감정 변화의 빈도와 강도를 반영하는데, 4차 모델에서는 이 값이 지나치게 작게 계산되는 경향이 있습니다. 이는 감정 변화의 세밀한 분석을 어렵게 만듭니다.

- **Significant Different Point(SDP) 식별의 부족**
  - 주요 감정 변동 구간을 식별하는 SDP의 민감도가 낮아, 중요한 감정 변화를 놓칠 수 있습니다. 특히 소설의 첫 부분에서 비정상적으로 큰 값이 감지되는 문제가 있습니다.



#### **(2) 5차 MODEL의 개선 및 추가기능**
5차 모델에서는 4차 모델의 한계점을 보완하고, 분석의 정밀도를 높이기 위해 다음과 같은 개선과 추가 기능을 도입하였습니다:

- **감정 극성의 긍정 편향 문제 해결**
  - 감정 사전을 재조정하고, 긍정적 단어와 부정적 단어의 균형을 맞추었습니다. 또한, 문맥 기반 가중치를 조정하여 소설 초반부에서 긍정적인 감정이 과도하게 반영되지 않도록 했습니다.
  - 기존의 문장 단위의 감정 분석에서 문맥 단위의 감정 분석도 추가하여 긍정 편향 문제를 일부 해결했습니다.

- **소설 극초반부에 EWMA 값의 불규칙한 변동 문제 해결**
  - EWMA 초기값 설정 방식을 조정하여, 첫 몇 개 문장은 단순 평균을 사용하고 그 이후부터 EWMA를 적용하였습니다. 추가로, 가우시안 스무딩을 적용하여 초반부의 불규칙한 변동을 줄였습니다.

- **length score 계산 방식 수정**
  - length score를 단순히 스케일 업하는 대신, 감정 변화의 빈도와 강도를 동시에 고려하는 새로운 척도를 도입하였습니다. 이를 통해 length score가 감정 변화의 정도를 더 정확히 반영할 수 있게 되었습니다.

- **Significant Different Point(SDP) 식별의 개선 및 Significant Volatility Point(SVP) 식별**
  - SDP의 민감도를 높이기 위해 임계값을 조정하고, 소설의 첫 부분에서 비정상적으로 큰 값이 감지되지 않도록 개선하였습니다. 또한, 가우시안 스무딩을 적용하여 감정 변동 구간을 더 정확히 식별할 수 있게 하였습니다.
  - Sentiment Volatility의 그래프에서 극대/극소점, 변화가 강한 지점을 설정하고 이에 대한 구간을 탐지하여 추가적으로 감정의 변화가 큰 지점을 찾는 새로운 방법을 고안해냈습니다.
  - 이로 인해 SDP, SVP를 합쳐 SFP(Significant Fluctuation Point)를 정의하고, 더욱 다각도로 감정 분석을 할 수 있도록 하였습니다.

- **감정 점수 통합 방식 개선**
  - 감정 점수 통합 시 각 감정의 점수에 가중치를 부여하여, 더 정교한 감정 점수를 계산합니다. 예를 들어, 'anger'는 -1, 'disgust'는 -0.5, 'fear'는 -0.7, 'joy'는 1, 'sadness'는 -1의 가중치를 부여합니다.

- **감정 변동성 분석 개선**
  - 감정 변동성(volatility) 분석 시 가우시안 스무딩에서의 곡률을 더 크게 하여, 감정 변화의 세부적인 부분을 더 정확히 시각화합니다.
  - x축을 문단의 수에 맞게 조정하여, 시각화된 그래프가 문단의 실제 수와 일치하도록 개선합니다.

- **Overall Sentiment Analysis의 점수 산정 방식 개선**
  - 기존의 적분 방법을 통한 점수 계산 방식에 약간의 불합리성이 있다고 판단하여, 점수의 산정 방식을 개선합니다.
  - 우선 중요하다고 생각하는 문단을 인식하고, 이러한 문단에 가중치를 두며, 문단의 개수와 전체 감정 극성 값의 합을 적절히 잘 조합하여 점수를 책정하는 방식으로 점수 산정 방식을 변경하였습니다.
  - 따라서 5차 MODEL에서는 문장 단위의 감정 분석을 문단 단위로 바꾸고, 중요한 문단에 대해서 가중치를 두는 중요한 기능이 추가되었다고 할 수 있겠습니다.

#### **(3) 5차 MODEL의 코드 (최종 코드)**

5차 모델의 더 자세한 사항을 알고 싶다면, **Sentiment Analysis of Literary Texts: Code and Explanation.ipynb** 파일을 확인하시면 됩니다.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import re
import pandas as pd
from scipy.ndimage import gaussian_filter1d
from transformers import pipeline

# 파일 경로 설정
file_path = '/경로/소설.txt'  # 파일 경로를 필요에 맞게 수정한다.

# 텍스트 파일 읽기
with open(file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# 텍스트를 문단 단위로 분할 (빈 줄을 기준으로)
paragraphs = text.split('\n\n')   # 문단 단위의 분할 기준도 소설 원문의 형식에 맞게 수정한다.
paragraphs = [paragraph for paragraph in paragraphs if paragraph]  # 빈 문단 제거

# 총 문단의 개수 계산
total_paragraphs = len(paragraphs)

# 감정 분석 모델 로드 (Hugging Face Transformers, BERT 기반 모델)
sentiment_analysis_pipeline = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

# 최대 입력 길이 설정
MAX_LENGTH = 512

# 감정 분석 수행 함수 정의
def get_sentiment_score(text, prev_score=None):
    if len(text) > MAX_LENGTH:
        text = text[:MAX_LENGTH]  # 입력 문장을 최대 길이로 자르기
    results = sentiment_analysis_pipeline(text)

    # 다중 감정 인식 및 점수 할당
    emotion_scores = {'anger': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0}
    for result in results:
        label = result['label']
        probability = result['score']

        if label == '1 star':
            emotion_scores['sadness'] += probability
        elif label == '2 stars':
            emotion_scores['fear'] += probability
        elif label == '3 stars':
            emotion_scores['disgust'] += probability
        elif label == '4 stars':
            emotion_scores['joy'] += probability
        elif label == '5 stars':
            emotion_scores['joy'] += 2 * probability

    # 감정 점수 통합
    score = (
        -1 * emotion_scores['anger'] +
        -0.5 * emotion_scores['disgust'] +
        -0.7 * emotion_scores['fear'] +
        1 * emotion_scores['joy'] +
        -1 * emotion_scores['sadness']
    )

    return score

# 긍정적 및 부정적 단어 리스트 정의
positive_words = [
    "happy", "joyful", "excellent", "good", "wonderful", "amazing", "great", "positive", "fortunate", "pleased",
    "delightful", "content", "cheerful", "satisfied", "thrilled", "ecstatic", "merry", "elated", "jubilant", "gleeful"
]

negative_words = [
    "sad", "terrible", "bad", "awful", "horrible", "negative", "unfortunate", "displeased", "angry", "fearful",
    "miserable", "depressed", "unhappy", "downcast", "gloomy", "dismal", "wretched", "heartbroken", "melancholy", "despair"
]

# 중요 문단을 인식하기 위한 키워드 리스트
important_keywords = [
    "conflict", "resolution", "climax", "turning point", "significant", "important", "crucial", "pivotal"
]

# 감정 분석 수행 함수 정의 (중요 문단 인식)
def get_sentiment_score(paragraph, prev_score=None):
    if len(paragraph) > MAX_LENGTH:
        paragraph = paragraph[:MAX_LENGTH]  # 입력 문장을 최대 길이로 자르기
    results = sentiment_analysis_pipeline(paragraph)

    # 문맥에 따라 긍정 또는 부정 가중치 적용
    context_weight = 1.0
    if any(word in paragraph for word in positive_words):
        context_weight = 1.2  # 긍정 문맥 가중치
    elif any(word in paragraph for word in negative_words):
        context_weight = 0.8  # 부정 문맥 가중치

    # 중요 문단 가중치 적용
    important_weight = 1.0
    if any(keyword in paragraph for keyword in important_keywords):
        important_weight = 1.5

    # 다중 감정 인식 및 점수 할당
    emotion_scores = {'anger': 0, 'disgust': 0, 'fear': 0, 'joy': 0, 'sadness': 0}
    for result in results:
        label = result['label']
        probability = result['score']

        if label == '1 star':
            emotion_scores['sadness'] += probability
        elif label == '2 stars':
            emotion_scores['fear'] += probability
        elif label == '3 stars':
            emotion_scores['disgust'] += probability
        elif label == '4 stars':
            emotion_scores['joy'] += probability
        elif label == '5 stars':
            emotion_scores['joy'] += 2 * probability

    # 감정 점수 통합 및 가중치 적용
    score = (
        -1 * emotion_scores['anger'] +
        -0.5 * emotion_scores['disgust'] +
        -0.7 * emotion_scores['fear'] +
        1 * emotion_scores['joy'] +
        -1 * emotion_scores['sadness']
    ) * context_weight * important_weight

    return score

# 첫 번째 문장은 이전 문장이 없으므로 먼저 처리
sentiment_analysis = [get_sentiment_score(paragraphs[0])]

# 나머지 문장 처리
for i in range(1, total_paragraphs):
    prev_score = sentiment_analysis[-1]
    sentiment_analysis.append(get_sentiment_score(paragraphs[i], prev_score))

# specific_size와 standard_num 계산
def calculate_specific_size(total_paragraphs, target_standard_num=20):
    best_specific_size = None
    closest_difference = float('inf')
    best_standard_num = None

    for specific_size in range(1, total_paragraphs + 1):
        standard_num = total_paragraphs // specific_size
        difference = abs(standard_num - target_standard_num)

        if difference < closest_difference:
            closest_difference = difference
            best_specific_size = specific_size
            best_standard_num = standard_num

        if closest_difference == 0:
            break

    return best_specific_size, best_standard_num

specific_size, standard_num = calculate_specific_size(total_paragraphs)

# 단순 누적 평균 계산
cumulative_avg = np.cumsum(sentiment_analysis) / (np.arange(total_paragraphs) + 1)

# 가중 누적 평균 계산 (EWMA)
ewma = pd.Series(sentiment_analysis).ewm(span=specific_size, adjust=False).mean()

# 이동 평균 계산 함수 정의
def moving_average(data, window_size):
    return [np.mean(data[i:i + window_size]) for i in range(0, len(data), window_size)]

# 세 가지 방법 모두 specific_size 만큼의 평균으로 통일
sentiment_avg = moving_average(sentiment_analysis, specific_size)
cumulative_avg_avg = moving_average(cumulative_avg, specific_size)
ewma_avg = moving_average(ewma, specific_size)

# 이동 평균의 최대값과 최소값 계산
moving_avg_max = max(sentiment_avg)
moving_avg_min = min(sentiment_avg)

# sigma 값을 동적으로 설정
sigma_value = max(0.1, min(1.0, 10.0 / np.sqrt(total_paragraphs)))  # 문장의 수에 따라 sigma 값을 조정
sentiment_avg_smooth = gaussian_filter1d(sentiment_avg, sigma=sigma_value)
cumulative_avg_smooth = gaussian_filter1d(cumulative_avg_avg, sigma=sigma_value)
ewma_avg_smooth = gaussian_filter1d(ewma_avg, sigma=sigma_value)

# X 좌표 설정
x_sentiment = np.arange(len(sentiment_avg)) * specific_size
x_cumulative = np.arange(len(cumulative_avg_avg)) * specific_size
x_ewma = np.arange(len(ewma_avg)) * specific_size

# 차이가 크게 나타나는 구간 식별
difference = np.abs(np.array(ewma_avg) - np.array(sentiment_avg))
std_dev_difference = np.std(difference)
threshold_difference = 1.2 * std_dev_difference  # 이동 표준 편차의 1.2배를 임계값으로 설정

significant_diff_indices = np.where(difference >= threshold_difference)[0]

# 오차 범위를 고려한 실제 문장 인덱스 계산
actual_diff_indices = []
for idx in significant_diff_indices:
    start_idx = max(0, idx * specific_size - specific_size // 2)
    end_idx = min(total_paragraphs, idx * specific_size + specific_size // 2)
    actual_diff_indices.append((start_idx, end_idx))

# 인접한 구간 합치기
merged_diff_indices = []
for start_idx, end_idx in actual_diff_indices:
    if not merged_diff_indices:
        merged_diff_indices.append([start_idx, end_idx])
    else:
        last_start, last_end = merged_diff_indices[-1]
        if start_idx <= last_end:
            merged_diff_indices[-1] = [last_start, max(last_end, end_idx)]
        else:
            merged_diff_indices.append([start_idx, end_idx])

# Sentiment Volatility 계산
volatility = np.diff(sentiment_avg_smooth)

# 가우시안 스무딩 적용 (곡률을 더 크게)
volatility_smooth = gaussian_filter1d(volatility, sigma=2)

# Sentiment Volatility에서 차이가 크게 나타나는 구간 식별
volatility_diff_indices = []
for i in range(1, len(volatility_smooth) - 1):
    if (volatility_smooth[i] > volatility_smooth[i-1] and volatility_smooth[i] > volatility_smooth[i+1]) or \
       (volatility_smooth[i] < volatility_smooth[i-1] and volatility_smooth[i] < volatility_smooth[i+1]):
        volatility_diff_indices.append(i)

# 오차 범위를 고려한 실제 문장 인덱스 계산
actual_volatility_indices = []
for idx in volatility_diff_indices:
    start_idx = max(0, idx * specific_size - specific_size // 2)
    end_idx = min(total_paragraphs, idx * specific_size + specific_size // 2)
    actual_volatility_indices.append((start_idx, end_idx))

# 인접한 구간 합치기
merged_volatility_indices = []
for start_idx, end_idx in actual_volatility_indices:
    if not merged_volatility_indices:
        merged_volatility_indices.append([start_idx, end_idx])
    else:
        last_start, last_end = merged_volatility_indices[-1]
        if start_idx <= last_end:
            merged_volatility_indices[-1] = [last_start, max(last_end, end_idx)]
        else:
            merged_volatility_indices.append([start_idx, end_idx])

# 그래프 그리기 (Sentiment Analysis)
plt.figure(figsize=(12, 6))

# 단순 누적 평균
plt.plot(x_cumulative, cumulative_avg_smooth, color='#DB4455', linestyle='--', label='Cumulative Average')

# 가중 누적 평균 (EWMA)
plt.plot(x_ewma, ewma_avg_smooth, color='#0E5CAD', linestyle='-', label='Exponential Weighted Moving Average')

# 이동 평균
plt.plot(x_sentiment, sentiment_avg_smooth, color='#623AA2', linestyle='-', label='Moving Average')

# S.D.P. 영역 추가
for start_idx, end_idx in merged_diff_indices:
    end_idx = min(end_idx, total_paragraphs)  # 소설의 끝을 넘지 않도록 조정
    plt.axvspan(start_idx, end_idx, color='lightgrey', alpha=0.5)

plt.xlabel('Paragraph Number')
plt.ylabel('Sentiment Polarity (-1 to 1)')
plt.title('Sentiment Analysis using Different Calculating Methods')
plt.axhline(0, color='grey', linestyle='--')  # 중립 감정선
plt.legend()

# 이동 평균의 최대값과 최소값을 그래프에 표시
plt.text(0.95, 0.10, f"max = {moving_avg_max:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')
plt.text(0.95, 0.05, f"min = {moving_avg_min:.2f}", fontsize=10, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

# 그래프 그리기 (Sentiment Volatility)
plt.figure(figsize=(12, 6))

# x축을 문장의 수에 맞게 조정
plt.plot(np.linspace(0, total_paragraphs, len(volatility_smooth)), volatility_smooth, color='#B4C99D', linestyle='-')
plt.axhline(0, color='grey', linestyle='--')  # y=0의 값에 회색 점선 추가

plt.xlabel('Paragraph Number')
plt.ylabel('Sentiment Change')
plt.title('Sentiment Volatility')

# S.V.P. 영역 추가
for start_idx, end_idx in merged_volatility_indices:
    end_idx = min(end_idx, total_paragraphs)  # 소설의 끝을 넘지 않도록 조정
    plt.axvspan(start_idx, end_idx, color='lightgrey', alpha=0.5)

# 길이 점수 출력 (그래프의 '네모 박스'의 우측 하단에 표시)
plt.text(0.95, 0.05, f"length score = {volatility_smooth.sum() * 1000:.2f}", fontsize=12, color='black', transform=plt.gca().transAxes, ha='right')

plt.show()

# 실제 문장 인덱스 기반 텍스트 분석
fig, ax = plt.subplots(figsize=(12, 2))
ax.text(0.5, 0.5, "<Significant Fluctuation Point>\n\nSignificant Different Point\n" + '\n'.join(
    [f"Index: {start} ~ {end}" for start, end in merged_diff_indices]) + "\n\nSignificant Volatility Point\n" + '\n'.join(
    [f"Index: {start} ~ {end}" for start, end in merged_volatility_indices]), horizontalalignment='center', verticalalignment='center', fontsize=12)
ax.axis('off')
plt.show()

# 문단별 긍정 및 부정 점수 분리
positive_scores = [score for score in sentiment_analysis if score > 0]
negative_scores = [score for score in sentiment_analysis if score < 0]

# 긍정 및 부정 점수 합산 (절댓값 사용)
positive_score_sum = sum(positive_scores)
negative_score_sum = abs(sum(negative_scores))

# 가중치 적용 (문단 수)
positive_weighted_sum = positive_score_sum * len(positive_scores)
negative_weighted_sum = negative_score_sum * len(negative_scores)

# 총합 계산
total_weighted_sum = positive_weighted_sum + negative_weighted_sum

# 백분율로 환산
positive_final_score = (positive_weighted_sum / total_weighted_sum) * 100
negative_final_score = (negative_weighted_sum / total_weighted_sum) * 100

# 막대 바 차트 시각화 코드
fig, ax = plt.subplots(figsize=(8, 6))

# 데이터 준비
categories = ['Positive Sentiment Score', 'Negative Sentiment Score']
scores = [positive_final_score, negative_final_score]
colors = ["#DC143C", "#392F31"]

# 막대 바 차트 그리기
bars = plt.bar(categories, scores, color=colors)

# 레이블 추가 (막대 바 위쪽에 위치하도록)
for bar, score in zip(bars, scores):
    yval = bar.get_height()
    plt.text(bar.get_x() + bar.get_width() / 2.0, yval + 0.1, f'{score:.2f} points', ha='center', va='bottom', color='black')

# y축 최대값 설정
max_score = max(positive_final_score, negative_final_score)
ymax = np.ceil(max_score * 1.2)

# y축에 숫자를 표시한 지점마다 x축과 평행한 점선 추가
ax.yaxis.grid(True, which='both', color='lightgrey', linestyle='--', linewidth=0.7)
ax.set_axisbelow(True)
ax.set_ylim(0, ymax)

# 제목 설정
plt.title('Overall Sentiment Analysis of the Novel')

plt.show()

## **II. 결론**



- 1차 모델부터 5차 모델까지의 발전 과정을 통해 감정 분석의 정확성과 정밀성을 크게 향상시켰습니다.
- 다양한 분석 방법과 감정 인식을 도입하여 소설의 감정 변화를 보다 상세하고 직관적으로 시각화할 수 있게 되었습니다.
- 이를 통해 문맥을 더 잘 파악하고, 긍정적 및 부정적 감정 변화를 정교하게 분석할 수 있습니다.



---

